In [1]:
# Import modules
from openpyxl.utils import column_index_from_string
from openpyxl.utils import get_column_letter
from pathlib import Path
import pandas as pd
import re
import threading
import time
import xlwings as xw

In [1]:
class ImportData:

    """
    Assists with importing data from excel into pandas.  Parse a list with a single excel file path or multiple 
    excel file paths.  Each file is used to create a single dataframe with all dataframes being appeded to a list. 
    The list of dataframes is then concatenated to create a primary dataframe that contains all of the imported data.
    
    -----------------------------------------------------------------------
    # Attributes:
    
        files_for_import - a list that can containing either a single file or multiple files.
    
        worksheet_index - the excel worksheet index (number index or name of worksheet).
    
        headings_data_row - row containing the column headings.
    
        right_data_column - right side column (letter) of the data.
    
        has_spacing_columns - boolean (remove every other column)
    
        column_to_copy - boolean (copy a column of data to act as dataframe index)
     
    -----------------------------------------------------------------------
        
    """
    
    def __init__(self,
                 files_for_import,
                 worksheet_index = 1, 
                 headings_data_row = 1,
                 index_column = None,
                 right_data_column = "R",
                 has_spacing_columns = True,
                 column_to_copy = False):
                
        # List of files to import
        self.files_for_import = files_for_import
        
        # Excel worksheet information
        self.worksheet_index = worksheet_index
        self.headings_data_row = headings_data_row
        self.index_column = index_column
        self.right_data_column = column_index_from_string(right_data_column) + 2
        self.has_spacing_columns = has_spacing_columns
        self.ignore_index = True if self.index_column is None else False
        self.column_to_copy = column_to_copy
        
        # Dataframes
        self.dataframes_list = []
        self.dataframe = []

        
    def make_dataframes_list(self):
        """ 
        Import excel file data into a dataframe.  Append each dataframe to a list.
        """
        print("Excel files imported:\n")
        for idx in range(len(self.files_for_import)):
            file_dataframe = pd.read_excel(io=self.files_for_import[idx], sheet_name=self.worksheet_index, header=self.headings_data_row, index_col=self.index_column)                                     
            
            # Remove spacing columns
            if self.has_spacing_columns:
                file_dataframe = file_dataframe.iloc[:,1:self.right_data_column:2]
            
            # Add each file dataframe to list
            self.dataframes_list.append(file_dataframe)
            
            # Print file dataframe information
            file_parts = self.files_for_import[idx].name
            print(f"{file_parts:40} {self.dataframes_list[idx].shape[0]:5} rows  {self.dataframes_list[idx].shape[1]:3} Columns")
        

    def make_primary_dataframe(self):
        """
        Vertically concatenate each dataframe to make a primary dataframe that contains all imported data.
        """
        self.dataframe =  pd.concat(self.dataframes_list, axis="rows", ignore_index=self.ignore_index)
        
        # Copy a column of data to act as datafram indexes
        if self.column_to_copy: self.copy_column_to_index()
        
    
    def copy_column_to_index(self):
        """
        Copy a column of data to use as the dataframe index.  
        """
        self.dataframe = self.dataframe.reset_index(drop=True).set_index(self.column_to_copy) 
        self.dataframe.insert(0, self.column_to_copy, self.dataframe.index)
        self.dataframe = self.dataframe.rename_axis("", axis='index')

        
    def make_dataframes(self):
        self.make_dataframes_list()
        self.make_primary_dataframe()
        
        # Release excel variables
        self.xlApp = None
        self.wb = None
        self.ws = None

In [4]:
class ExportData:
    """
    Assists with exporting data from pandas into excel. 
    
    -----------------------------------------------------------------------
    # Attributes:
    
        export_dataframe - the dataframe for export.
    
        dataframe_name  - the name to be added to the excel worksheet.
    
        excel_target_cell - the top left worksheet cell in which the dataframe should be be pasted.
    
        title_font - font to be used for the column headings.
    
        content_font - font to be used for the main table data.
    
        large_font_size - font size to be used for the column headings.
        
        small_font_size - font size to be used for the main table data.
     
    -----------------------------------------------------------------------
        
    """
    
    def __init__(self, \
                 export_dataframe, \
                 dataframe_name, \
                 excel_target_cell = "B3", \
                 title_font = "Lato", \
                 content_font = "Source Sans Pro", \
                 large_font_size = 16, \
                 small_font_size = 11):
        
        # Dataframe
        self.dataframe = export_dataframe
        self.dataframe_name = dataframe_name
        
        # Connect to excel 
        self.xlApp = xw.apps.active
        if self.xlApp is None: raise Exception("Excel must be open to continue!")
        
        # Test workbook connection
        self.connection_tested = False
        t1 = threading.Thread(target=self.connection_timer)
        t2 = threading.Thread(target=self.test_connection)
        t1.start(), t2.start()
        t1.join(), t2.join()
        
        # Connect to workbook
        if self.xlApp is None: raise Exception("Excels' data entry mode successfully deactivated.  Please rerun code.") 
        self.wb = xw.books.active
    
        # Set range variables
        self.top_left_cell = excel_target_cell
    
        # Font types and font sizes
        self.title_font =  title_font
        self.content_font = content_font
        self.large_font_size = large_font_size
        self.small_font_size = small_font_size
        
    
    def connection_timer(self):
        """
        Times out the attenpt to connect to excel when the test connection hangs.  This occurs when excel
        is in data entry mode (cursor flashing in a cell). 
        """
        time.sleep(0.1)
        if not self.connection_tested: 
            self.xlApp = None
            raise Exception("Excel is in data entry mode! Go to excel and press the 'esc' key to deactivate.")
            
            
    def test_connection(self):
        """
        Quickly attempts to connect to excel to check a connection can be successfully established.
        """
        # Try connecting to the active workbook
        wb = xw.books.active
        wb = None
        self.connection_tested = True
    

    def worksheet_connection(self): 
        """
        Select requested worksheet or make a new worksheet.
        """
        try:
            self.ws = self.wb.sheets.add(name=self.dataframe_name, after=self.wb.sheets.count)
        except Exception:
            self.wb.sheets[self.dataframe_name].delete()
            self.ws = self.wb.sheets.add(name=self.dataframe_name, after=self.wb.sheets.count)
            

    def export_data(self):
        """
        Converts a pandas series to a dataframe to enable export. 
        """
        try:
            self.dataframe.shape[1]
        except IndexError:
            self.dataframe = self.dataframe.to_frame()
        finally:
            self.ws.range("A2").options(pd.DataFrame, header=1, index=False, expand='table').value = self.dataframe
            
            for col_indx in range(self.dataframe.shape[1], 0, -1):
                    self.ws.range(1, col_indx).api.EntireColumn.Insert()
                    
                    
    def data_coordinates(self):
        """
        Set key data coordinates for use in formatting excel worksheet appearance. 
        """
        row_count = self.dataframe.shape[0]
        col_count = (self.dataframe.shape[1] *2)
        
        top_left_cell = ("B3")
        bottom_right_cell = f"{get_column_letter(col_count)}{row_count + 2}"
        left_col = "B"     
        right_col = get_column_letter(col_count)
        top_row = 3  
        bottom_row = 3 + row_count
        
        # Script coordinate variables
        self.headings_bar = self.ws.range(f"({left_col}2:{right_col}2)")
        self.headings_data = self.ws.range(f"({left_col}2:{right_col}2)")                     
        self.table_data = self.ws.range(f"({top_left_cell}:{bottom_right_cell})")
        
    
    def column_widths(self):
        """
        Dataframe column headings are mapped to column widths.  The widths are applied to the 
        columns in excel.  This allows the dataframe columns to exported in any order whilst ensuring 
        the approprite column width is maintained.
        """
        # All possible column headings and output column widths 
        all_column_widths = {'Key':                15.00, \
                             'Author':             30.00, \
                             'First Author':       20.00, \
                             'Title':              45.29, \
                             'Title Lower':        45.29, \
                             'Year':                7.43, \
                             'Journal':            51.71, \
                             'Name of Database':   42.86, \
                             'Abstract':           77.57, \
                             'Type of Research':   77.57, \
                             'Notes':             124.71, \
                             'Query Duplicate':    30.00, \
                             'Flag Number':        30.00, \
                             "Status":             42.86, \
                             "Exclusion Reason One (Abstract)":  50.00, \
                             "Exclusion Reason Two (Abstract)":  50.00, \
                             "Exclusion Reason Three (Abstract)":  50.00, \
                             "Exclusion Reason (Full-Article)":  50.00, \
                             "key Number": 120.75, \
                             "Key": 22.29, \
                             "First Author's Name": 39.86, \
                             "Publication Title": 71.43, \
                             "Journal Name": 41.71, \
                             "Year of Publication": 41.43, \
                             "Country of Conduct": 39.14, \
                             "Type of Evidence": 56.14, \
                             "Study Design": 23.86, \
                             "Type of Cancer": 33.57, \
                             "Sample Size": 23.29, \
                             "Num. Males": 28.43, \
                             "Num. Females": 29.71, \
                             "Average Age": 34.57, \
                             "Primary Staging": 26.57, \
                             "Secondary Staging": 33.29, \
                             "Imaging Modality": 32.57, \
                             "Extraction Method": 42.14, \
                             "Longitudinal Study": 36.57, \
                             "Muscle Seg.": 33.86, \
                             "SAT Seg.": 29.14, \
                             "VAT Seg.": 29, \
                             "Total Psoas Area (TSA)": 47.43, \
                             "Total Psoas Index (TSI)": 50, \
                             "Skeletal Muscle Area (SMA)": 73, \
                             "Skeletal Muscle Index (SMI)": 50, \
                             "Skeletal Muscle Density (Myosteatosis)": 75.29, \
                             "Skeletal Muscle Gauge (SMG)": 69.71, \
                             "Lean Muscle Mass (LMM)": 69.14, \
                             "Bone Mineral Density (BMD)": 58, \
                             "Psoas Muscle Density (Myosteatosis)": 66.29, \
                             "Intermuscular Adipose Tissue (IMAT)": 80.86, \
                             "Subcutaneous Adipose Tissue (SAT)": 85.57, \
                             "Visceral Adipose Tissue (VAT)": 82.43, \
                             "Total Adipose Tissue (TAT)": 63.43, \
                             "Relative Visceral Fat Area (rVFA)": 65.29, \
                             "Subcutaneous Adipose Tissue Index (SATI)": 65.86, \
                             "Visceral Adipose Tissue Index (VATI)": 75.57, \
                             "Total Fat Mass Index (TFMI)": 67, \
                             "Visceral Adipose Tissue Density": 59.14, \
                             "Subcutaneous Adipose Tissue Density": 62.71, \
                             "Total Adipose Tissue Density": 66, \
                             "Sarcopenic Obesity / Non-Sarcopenic Obesity": 87.71, \
                             "Sarcopenia / Non-Sarcopenia": 70, \
                             "Sarcopenia and Adipopenia": 50, \
                             "Multivariate Analysis": 50, \
                             "Assessment Outcome 1": 50, \
                             "Assessment Outcome 2": 50, \
                             "Assessment Outcome 3": 50, \
                             "Assessment Outcome 4": 50, \
                             "Study Findings ": 151.86, \
                             "AI Summary (200 Words)": 195.29, \
                             }
    

        # Add blank spacing-headings to the list of dataframe headings
        dataframe_headings = list(self.dataframe.columns)
        for idx in range(len(dataframe_headings), -1, -1):
            dataframe_headings.insert(idx, "")
        
        # Generate a dataframe specific list of column widths
        widths = [all_column_widths.get(each_heading, '0.5') for each_heading in dataframe_headings]
        
        # Apply column widths to exel worksheet
        for idx in range(0, len(dataframe_headings)):
            col_indx = idx +1
            self.ws.range(5, col_indx).column_width = widths[idx]
        
        
    def sheet_formatting(self): 
        """
        Excel worksheet formatting.
        """
        # Headings bar
        self.ws.range("1:1").row_height = 5
        self.headings_data.row_height = 24
        self.headings_bar.api.Interior.Color = 15132135 # mapei_grey

        # Headings text
        self.headings_data.font.name = "Lato"
        self.headings_data.font.size = 16
        self.headings_data.font.bold = True
        self.headings_data.api.HorizontalAlignment = -4131  # xlHAlignLeft	
        self.headings_data.api.VerticalAlignment = -4108 # xlVAlignCenter	
       
        # Data table text
        self.table_data.font.name = "Source Sans Pro"
        self.table_data.api.HorizontalAlignment = -4131  # xlHAlignLeft
        self.table_data.api.VerticalAlignment = -4160 # xlVAlignTop
        self.table_data.wrap_text = True
        self.table_data.row_height = 18
        
        # Freeze titlebar and headings rows
        self.ws.range("3:3").select()
        self.xlApp.api.ActiveWindow.FreezePanes = True
        self.ws.range("B3").select()  
    
    
    def export_to_excel(self):
        self.worksheet_connection()
        self.export_data()
        self.data_coordinates()
        self.column_widths()
        self.sheet_formatting() 
        
        # Release excel variables
        self.xlApp = None
        self.wb = None
        self.ws = None